# Análisis de Ddatos
## TP N 1
----
### Grupo N
- Aviani, José
- Díaz, José Luis
- Silvera, Ricardo

---
## Introducción

Para este trabajo elegimos el el dataset Precios Claros – Base SEPA, perteneciente al “Sistema Electrónico de Publicidad de Precios Argentinos (SEPA)" (https://datos.gob.ar/), el cual reúne los precios de comercios minoristas (grandes establecimientos) de más de 70 mil productos en toda la Argentina. Particularmente para este trabajo, seleccionamos el set de datos del establecimiento **Carrefour** ya que era el de mayor tamaño, lo cual es deseable como entrada en un problema de aprendizaje de máquina.
A continuación realizamos el análisis exploratorio de los datos y finalizamos con las conclusiones obtenidas del trabajo.


### Importación de librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


## Análisis Exploratorio de los Datos

### Carga y comprensión de los datos
Los datos están separados en 3 archivos
- comercio
- sucursales
- productos

Cada uno contiene información específica sobre la entidad que indica su nombre. Vamos a investigar a cada uno para comprender la información que contienen y como está organizada y unificar todos los datos que nos interesen en un único dataframe. 

### Comercio

In [ ]:
comercio=pd.read_csv('./dataset/comercio.csv',delimiter='|')
# La ultima linea del dataset es solamente la fecha de actualizacion, por eso la borramos
comercio = comercio.iloc[:-1]
comercio.columns

El dataframe **comercio** contiene solo 3 filas con información, mucha de la cuales son repetitiva y constante para nuestro caso que solo estudiaremos un solo comercio.

- `id_comercio`: Es el identificador de comercio, como trabajaremos solo con Carefour este data será constante en todo el dataset.

- `id_bandera`: Es de tipo numérico,  identifica el tipo de comercio cuya discreción en texto está en el campo comercio_bandera_nombre 
- `icomercio_cui`: Es de tipo numérico, contiene el numero de CUIT del comercio, constante para nuestro caso. 
- `comercio_razon_social`:  Es de tipo texto, contiene el la Razón Social del comercio, es constante para nuestro caso.
- `comercio_bandera_nombre`: Contiene la descripción del tipo de comercio (Hipermercado Carrefeour, Express, Market). es la descripción del valor numérico del campo id_bandera.
- `comercio_ultima_actualizacion`: Contiene la fecha y hora de la actualización del archivo. Es constante para nuestro caso ya que trabajaremos con un único archivo.
- `comercio_version_sepa`: Es el numero de versión que indica la versión del modelo de datos del paquete SEPA.


Las unicas columnas que tienen sentido en este caso son `id_bandera` y `comercio_bandera_nombre`, que nos permitiran entender el dimensión del tipo de sucursal.

In [ ]:
comercio.head()

In [ ]:
# Nos quedamos con las columnas relevantes
comercio.drop(columns=['id_comercio', 'comercio_cuit', 'comercio_razon_social', 'comercio_bandera_url','comercio_ultima_actualizacion', 'comercio_version_sepa'], inplace=True)
# Hacemos un cambio de tipo a entero. Ya que es un identificador, tiene mas sentido que un float.
comercio['id_bandera'] = comercio['id_bandera'].astype(int)
comercio.head()

### Sucursales

In [ ]:
sucursales=pd.read_csv('./dataset/sucursales.csv',delimiter='|')
# La ultima linea del dataset es solamente la fecha de actualizacion, por eso la borramos
sucursales = sucursales.iloc[:-1]
sucursales.info()

In [ ]:
# Los horarios de atencion no parecerian tener alguna utilidad a priori 
sucursales.drop(
    columns = [
        'sucursales_lunes_horario_atencion',
        'sucursales_martes_horario_atencion',
        'sucursales_miercoles_horario_atencion',
        'sucursales_jueves_horario_atencion',
        'sucursales_viernes_horario_atencion', 
        'sucursales_sabado_horario_atencion',
        'sucursales_domingo_horario_atencion',
    ], inplace=True)

In [ ]:
sucursales['id_comercio'].unique()

In [ ]:
# Como id_comercio hace referencia al id de "comercio.csv", al ser unico para todas las sucursales no aporta nada de valor
sucursales.drop(columns = ['id_comercio'], inplace=True)


In [ ]:
sns.countplot(data=sucursales, x='sucursales_tipo', color="lightblue")

El campo `sucursales_tipo` es una categorización de la sucursal según las características de su línea de cajas, definido por la cantidad de líneas de caja:
* Hipermercado: más de 15 cajas. 
* Supermercado: entre 4 y 15 cajas. 
* Autoservicio: entre 1 y 3 cajas

In [ ]:
# Tiene mas sentido que sea category
sucursales['sucursales_tipo'] = sucursales['sucursales_tipo'].astype('category')

# Podemos agregar el nombre de bandera a nuestras sucursales.
sucursales['id_bandera'] = sucursales['id_bandera'].astype(int)
sucursales = pd.merge(sucursales, comercio, on='id_bandera', how='outer')
sucursales['id_sucursal'] = sucursales['id_sucursal'].astype(int)


In [ ]:
# Graficamos que cantidad de cajas suele haber en las distintas banderas
relationship_table = pd.crosstab(sucursales['comercio_bandera_nombre'], sucursales['sucursales_tipo'])

# Mejor orden para las columnas
relationship_table = relationship_table[['Autoservicio', 'Supermercado', 'Hipermercado']]
relationship_table.rename(
    columns={
        'Autoservicio': 'entre 1 y 3', 
        'Supermercado': 'entre 4 y 15', 
        'Hipermercado': 'mas de 15'
    }, inplace=True)

# Mejor orden para las filas
relationship_table = relationship_table.reindex(['Express', 'Market', 'Hipermercado Carrefour' ])

plt.figure(figsize=(8, 6))
sns.heatmap(relationship_table, annot=True, fmt="d", cmap="YlGnBu", linewidths=.5)
plt.title('Heatmap tipo de bandera vs tipo de sucursal')
plt.xlabel('Cantidad de cajas')
plt.ylabel('Bandera')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()


In [ ]:
suma_faltantes_barrios = sucursales['sucursales_barrio'].isna().sum()
suma_faltantes_obs= sucursales['sucursales_observaciones'].isna().sum()

total = len(sucursales) 
print(f"Porcentaje de campos faltantes en sucursales_barrio: {(suma_faltantes_barrios/total)*100:.2f}%")
print(f"Porcentaje de campos faltantes en sucursales_observaciones: {(suma_faltantes_obs/total)*100:.2f}%")

In [ ]:
sucursales['temp_numeric'] = pd.to_numeric(sucursales['sucursales_numero'], errors='coerce')
problematic_rows = sucursales[sucursales['temp_numeric'].isna()]

problematic_rows[['sucursales_numero', 'sucursales_calle', 'sucursales_observaciones', 'sucursales_localidad', 'sucursales_latitud', 'sucursales_longitud']]

Algunas verificaciones usando google maps:

| Numero de fila | sucursal_calle + sucursales_observaciones | que dice google maps? |
| --- | --- | --- |
| 45 | Av. Avalos y lavalle | Av. Avalos 751 |
| 65 | Av. Gaona y Graham Bell | No registra numero en google  |
| 91 | Panamericana Ramal Pilar	Km 54 | No registra numero en google |
| 110 | Av. Díaz Vélez Esquina Medrano | Av. Díaz Vélez 3901 |




In [ ]:
# Reflejando algunos cambios
sucursales.loc[45, 'sucursales_numero'] = 751
sucursales.loc[110, 'sucursales_numero'] = 3901
sucursales.drop(columns = ['temp_numeric'], inplace=True)

# Ya tenemos el nombre de la bandera, con eso es suficiente
sucursales.drop(columns = ['id_bandera'], inplace=True)



In [ ]:
sucursales.head(10)

El archivo contiene información sobre las 563 sucursales del comercio

Nos quedamos con los valores de las sucursales que georeferencia el establecimiento y que nos dan una orden de magnitud de su tamaño:

-- NOTA --::: Deberiamos borrar sucursales nombre???

#### Identificadores
- `id_sucursal`: Es de tipo numérico, es el identificador de la sucursal

#### Tamaño
- `sucursales_tipo`: Es de tipo texto, indica el tipo de establecimiento. Relacionado con la cantidad de cajas.
- `comercio_bandera_nombre`:  'Hipermercado Carrefour', 'Express' o 'Market'

#### Ubicación
- `sucursales_calle`: Es de tipo texto, representa el nombre de la calle donde está ubicada la sucursal.
- `sucursales_numero`: Es de tipo numérico o texto, indica el número de la dirección de la sucursal.
- `sucursales_latitud`: Es de tipo numérico, representa la coordenada de latitud de la ubicación geográfica de la sucursal.
- `sucursales_longitud`: Es de tipo numérico, representa la coordenada de longitud de la ubicación geográfica de la sucursal.
- `sucursales_observaciones`: Es de tipo texto, contiene observaciones o notas adicionales sobre la sucursal.
- `sucursales_codigo_postal`: Es de tipo texto o numérico, representa el código postal correspondiente a la dirección de la sucursal.
- `sucursales_localidad`: Es de tipo texto, indica la localidad o ciudad donde se encuentra la sucursal.
- `sucursales_barrio`: Es de tipo texto, indica el barrio donde se encuentra ubicada la sucursal.
- `sucursales_provincia`: Es de tipo texto, contine un código que indica la provincia a la que pertenece la sucursal.



### Productos

In [ ]:
productos=pd.read_csv('./dataset/productos.csv.gz',delimiter='|',low_memory=False, compression='gzip')

In [ ]:
productos.info()

In [ ]:
productos.head()

El archivo contiene 2.835.685 filas con información de los productos.

#### Itentificadores

-  `id_comercio ` : Es de tipo numérico, representa el identificador único del comercio al que pertenece el producto. Es constante para nuestro caso.

-  `id_bandera `: Se refiere al tipo de comercio, y está relacionado con el campo  `id_bandera ` del archivo de comercio.
-  `id_sucursal `: Es de tipo numérico, contiene el código de la sucursal específica dentro del comercio, se relaciona con el campo de mismo nommbre del archivo sucursales.
-  `id_producto `: Es de tipo numérico, es el identificador único del producto para el comercio.
-  `productos_ean `: Es de tipo numérico, representa el código EAN (European Article Number) del producto, utilizado como código de barras.
-  `productos_descripcion `: Es de tipo texto, contiene la descripción comercial del producto.
-  `productos_marca`: Es de tipo texto, indica el nombre de la marca comercial del producto.

#### Unidades de medida y cantidades

-  `productos_cantidad_presentacion `: Es de tipo numérico, indica la cantidad del producto contenida en la unidad.

-  `productos_unidad_medida_presentacion `: Es de tipo texto, representa la unidad de medida de la presentación del producto.
-  `productos_cantidad_referencia `: Es de tipo numérico, indica la cantidad de producto usada como base para calcular el precio de referencia.
-  `productos_unidad_medida_referencia `: Es de tipo texto, representa la unidad de medida de la cantidad de referencia.

#### Precios 
-  `productos_precio_lista` : Es de tipo numérico, representa el precio de lista del producto, es decir, el precio regular sin promociones.

-  `productos_precio_referencia `: Es de tipo numérico, corresponde al precio de referencia.

#### Promociones

-  `productos_precio_unitario_promo1 `: Es de tipo numérico, indica el precio unitario del producto en la primera promoción (si existe).

-  `productos_leyenda_promo1 `: Es de tipo texto, contiene la leyenda o descripción asociada a la primera promoción.
-  `productos_precio_unitario_promo2 `: Es de tipo numérico, indica el precio unitario del producto en la segunda promoción (si existe).
-  `productos_leyenda_promo2 `: Es de tipo texto, contiene la leyenda o descripción asociada a la segunda promoción.

In [ ]:
#Se elimina la columna id_comercio
productos.drop(columns='id_comercio')

Descartamos los datos del dataframe **comercio** ya que sus datos son constantes, excepto `id_bandera` y `comercio_bandera_nombre` que no son redundantes para nuestro estudio, ya que contamos con el campo `id_bandera` en el archivo sucursales.
Unimos el dataframe de **productos** y **sucursales** para dejarlos en un solo objeto y poder analizar toda la información junta.

In [ ]:
datos= pd.merge(productos,sucursales,on='id_sucursal')
datos.head()

### Resumen estadístico general



In [ ]:
datos.describe()

### Revisión de valores faltantes 



### Análisis de tipos de variables



### Distribuciones
Histogramas de variables numéricas

Gráficos de barras para variables categóricas

Boxplots para detectar outliers

In [ ]:
datos['productos_precio_lista'].hist(bins=30)
sns.boxplot(x=datos['productos_precio_lista'])

### Relación entre variables
Matriz de correlación (df.corr())

Gráficos de dispersión (sns.scatterplot())

Agrupaciones y comparaciones por categorías (groupby())

### Detección de outliers 

### Segmentación y agrupamientos

---
## Conclusiones